In [18]:
from EduData import get_data
import os

if not os.path.exists('../../data/anonymized_full_release_competition_dataset/anonymized_full_release_competition_dataset.csv'):
    get_data("assistment-2017", "../../data")

downloader, INFO http://base.ustc.edu.cn/data/ASSISTment/anonymized_full_release_competition_dataset.zip is saved as ..\..\data\anonymized_full_release_competition_dataset.zip


downloader, INFO ..\..\data\anonymized_full_release_competition_dataset.zip is unzip to ..\..\data\anonymized_full_release_competition_dataset


In [19]:
import pandas as pd
import tqdm

data = pd.read_csv(
    '../../data/anonymized_full_release_competition_dataset/anonymized_full_release_competition_dataset.csv',
    usecols=['startTime', 'timeTaken', 'studentId', 'skill', 'problemId', 'correct','attemptCount', 'hintCount']
).dropna(subset=['skill', 'problemId']).sort_values('startTime')

data.timeTaken = data.timeTaken.astype(int)

In [20]:
skills = data.skill.unique().tolist()
problems = data.problemId.unique().tolist()
at = data.timeTaken.unique()

# question id from 1 to #num_skill
skill2id = { p: i+1 for i, p in enumerate(skills) }
problem2id = { p: i+1 for i, p in enumerate(problems) }
at2id = { a: i for i, a in enumerate(at) }

print("number of skills: %d" % len(skills))
print("number of problems: %d" % len(problems))
print("number of answer time: %d" % len(at))

number of skills: 102
number of problems: 3162
number of answer time: 1326


In [21]:
import numpy as np

it = set()
# calculate interval time
for u in data.studentId.unique():
    startTime = np.array(data[data.studentId == u].startTime)
    for i in range(1, len(startTime)):
        item = (startTime[i] - startTime[i - 1]) // 60
        if item > 43200:
            item = 43200
        it.add(item)

it2id = { a: i for i, a in enumerate(it) }
print("number of interval time: %d" % len(it))

number of interval time: 2839


In [22]:
# problems to skills
problem2skill = {}
for s, p in zip(np.array(data.skill), np.array(data.problemId)):
    problem2skill[problem2id[p]] = skill2id[s]
with open('../../data/anonymized_full_release_competition_dataset/problem2skill', 'w', encoding='utf-8') as f:
    f.write(str(problem2skill))

In [23]:
def parse_all_seq(students):
    all_sequences = []
    for student_id in tqdm.tqdm(students, 'parse student sequence:\t'):
        student_sequence = parse_student_seq(data[data.studentId == student_id])
        all_sequences.extend([student_sequence])
    return all_sequences


def parse_student_seq(student):
    seq = student
    s = [skill2id[q] for q in seq.skill.tolist()] # 学生行为记录中的 skill 列（技能ID）转换为相应的 skill2id 映射，生成技能ID列表 s
    a = seq.correct.tolist()
    p = [problem2id[p] for p in seq.problemId.tolist()] # 将 problemId 列中的题目ID映射到 problem2id，生成题目ID列表 p
    it = [0]   # 初始化时间间隔列表 it，其中第一个时间间隔为0
    startTime = np.array(seq.startTime) # 将 startTime 列（表示每道题目开始作答的时间）转换为Numpy数组，方便处理时间差。
    for i in range(1, len(startTime)): 
        item = (startTime[i] - startTime[i - 1]) // 60
        if item > 43200:
            item = 43200
        it.append(it2id[item])  # 将时间差映射为 it2id，并添加到 it 列表中。
    at = [at2id[int(x)] for x in seq.timeTaken.tolist()] # 
    # （timeTaken）映射到 at2id，生成回答时间ID列表 at。
    return s, a, p, it, at


sequences = parse_all_seq(data.studentId.unique())

parse student sequence:	: 100%|██████████| 1709/1709 [00:11<00:00, 152.59it/s]


In [24]:
from sklearn.model_selection import train_test_split, KFold

# split train data and test data
train_data, test_data = train_test_split(sequences, test_size=.2, random_state=10)
train_data = np.array(train_data)
test_data = np.array(test_data)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (1367, 5) + inhomogeneous part.

In [27]:
def sequences2l(sequences, trg_path):
    with open(trg_path, 'a', encoding='utf8') as f:
        for seq in tqdm.tqdm(sequences, 'write data into file: %s' % trg_path):
            s_seq, a_seq, p_seq, it_seq, at_seq = seq
            seq_len = len(s_seq)
            f.write(str(seq_len) + '\n')
            f.write(','.join([str(s) for s in s_seq]) + '\n')
            f.write(','.join([str(a) for a in a_seq]) + '\n')
            f.write(','.join([str(p) for p in p_seq]) + '\n')
            f.write(','.join([str(i) for i in it_seq]) + '\n')
            f.write(','.join([str(a) for a in at_seq]) + '\n')

from sklearn.model_selection import KFold

# split into 5 folds
kfold = KFold(n_splits=5, shuffle=True, random_state=10)
idx = 0
for train_data_1, valid_data in kfold.split(train_data):
    # 使用列表推导式根据索引提取子集
    train_subset = [train_data[i] for i in train_data_1]
    valid_subset = [train_data[i] for i in valid_data]
    
    sequences2l(train_subset, '../../data/anonymized_full_release_competition_dataset/train' + str(idx) + '.txt')
    sequences2l(valid_subset, '../../data/anonymized_full_release_competition_dataset/valid' + str(idx) + '.txt')
    
    idx += 1

# 处理测试数据
sequences2l(test_data, '../../data/anonymized_full_release_competition_dataset/test.txt')


write data into file: ../../data/anonymized_full_release_competition_dataset/train0.txt: 100%|██████████| 1093/1093 [00:01<00:00, 695.26it/s]
write data into file: ../../data/anonymized_full_release_competition_dataset/valid0.txt: 100%|██████████| 274/274 [00:00<00:00, 1062.35it/s]
write data into file: ../../data/anonymized_full_release_competition_dataset/train1.txt: 100%|██████████| 1093/1093 [00:01<00:00, 813.15it/s]
write data into file: ../../data/anonymized_full_release_competition_dataset/valid1.txt: 100%|██████████| 274/274 [00:00<00:00, 734.24it/s]
write data into file: ../../data/anonymized_full_release_competition_dataset/train2.txt: 100%|██████████| 1094/1094 [00:01<00:00, 899.51it/s] 
write data into file: ../../data/anonymized_full_release_competition_dataset/valid2.txt: 100%|██████████| 273/273 [00:00<00:00, 1102.27it/s]
write data into file: ../../data/anonymized_full_release_competition_dataset/train3.txt: 100%|██████████| 1094/1094 [00:01<00:00, 978.12it/s] 
write da